In [61]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_cohere import CohereEmbeddings
import cassio
import os, dotenv
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
dotenv.load_dotenv()

True

In [34]:
pdf_doc = PyPDFLoader("National AI Policy Consultation Draft.pdf").load()
len(pdf_doc)

41

In [36]:
doc_chunks = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(pdf_doc)
print(doc_chunks[10])
len(doc_chunks)

Document(
    metadata={'source': 'National AI Policy Consultation Draft.pdf', 'page': 4},
    page_content='5 \n Foreword  \nThrough its continued efforts toward  the early realization of the Digital 
Pakistan vision of the Government \nof Pakistan, the Ministry of IT & Telecom is committed to providing  its people
with timely and equal access \nto opportunities by stimulating a culture of innovation through an overarching de 
velopmental agenda \norchestrated to embrace cutting -edge technologies such as Artificial Intelligence efficiently
and \nresponsibly.   \nIn this regard, the  Artificial Intelligence  (AI) Policy  is a pivotal milestone for  
transforming Pakistan into a  \nknowledge -based economy as it spells out a national strategy  to establish an 
ecosystem necessary for AI \nadoption by harnessing an agile framework for addressing different aspects of unique 
user journeys \nencompassing different market horizontals and industry verticals by ensuring responsible use of AI.
\nFurthermore, the policy aims to go beyond the meagre approach of adopting technology to fundamentally'
)

157

In [66]:
ASTRA_DB_APPLICATION_TOKEN = os.getenv('ASTRA_DB_APPLICATION_TOKEN')
ASTRA_DB_ID = os.getenv('ASTRA_DB_ID')

In [71]:
cassio.init(
    database_id=ASTRA_DB_ID,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

In [72]:
import os
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["COHERE_API_KEY"] = os.getenv('COHERE_API_KEY')

## LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
## Embeddings
embeddings = CohereEmbeddings()

In [73]:
astra_vector_store = Cassandra(embedding=embeddings,
                               table_name="astra_rag",
                               )

In [74]:
astra_vector_store.add_documents(doc_chunks)

['d4c8b7fa79d3407fb87dcfa6c788458b',
 '0f15e3bfee304bc6afd5f541358c5145',
 '9e6a43b00188476aadd1732dd784ce9d',
 'c75b94359d4b45768ab8bcf15999012b',
 '97baa03275864b7c8be4c2fe1fda96ee',
 'aea559d186ba4b37bf29fe1d1e5ef60f',
 '9f0d638dec9d4c8496590813e2c9b35e',
 '541ed555469941c298e35be18fe0f962',
 '3a24241bd4564587ba7c6b119291547e',
 'c520fc57cca34356bad52fb4a5cb8a72',
 'e636ee1030204f199d9ae536055b8000',
 '5f779b2acc2447289c063be627efafab',
 'a2aeb64fc7df4506b4f33808bf90e018',
 '19f5661d3aed49d8b4740cb1bc72c8a3',
 '43ba4fc8725f4f83a0472033ade5a2cc',
 '2f3b144526ad4ce4b4498e35486232e0',
 '655d22a87c8144f996181c69c003cff4',
 'e2d0a6b580e34795ad19dc3e1622312d',
 '19cfbf0cd27046cbb6ac1b3b954a7027',
 'd2163bfb13904f80ab6a98a6b6d8cf0b',
 'db3430b567bf44da9e93be07cfcbb6fc',
 'ecc9b4a6cd87456585795170210ef623',
 '8779effd7f484f1faff536636b6ca35c',
 '3ba074bc57e84cc1b6d0b0e675944cee',
 '0fed01bb711c472a9bdd40b795a3df35',
 'f666674bfd1a4877812768efea8baeae',
 '1cdaadb1be244a5aa749c55f5d393047',
 

In [75]:
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

In [76]:
from rich import print

response = astra_vector_index.query("What is National Artificail Intelligence Policy?", llm=llm)
print(response)

The provided text is a draft of Pakistan's National Artificial Intelligence Policy. It outlines the government's 
plan to stimulate growth in AI across the country. 

The policy focuses on several key areas:

* **AI Market Enablement:** This includes establishing a National AI Fund (NAIF) to support research and 
development in AI and related technologies.
* **Center of Excellence in AI & Allied Technologies:**  This aims to create a hub for expertise and innovation in 
AI.
* **Catalyzing Social Development through AI:**  This focuses on using AI to address social challenges and improve 
the lives of citizens.
* **Data and Computational Infrastructure:** This emphasizes the need for robust data infrastructure to support AI 
development and deployment.

The policy aims to empower Pakistan to leverage AI for economic growth, social progress, and national development.

In [77]:
query = "What are the objectives of National AI Fund?"

retrieved_docs = astra_vector_store.similarity_search_with_relevance_scores(query=query, k=3)
print(retrieved_docs)

[
    (
        Document(
            metadata={'page': '15.0', 'source': 'National AI Policy Consultation Draft.pdf'},
            page_content='to establish a National AI Fund with following objectives . \nI. In accordance with the 
stipulations of clauses 33D (II) & (III) of the Telecommunication Re -\norganization Act 1996 (amended 2006),  the 
Ministry of IT & Telecom, while exercising its right to \nissue policy directives, shall direct the Research & 
Development Fund (Ignite – Technology Fund) \nto allocate a part (not less than 30%) of its funds to NAIF on a 
perpetual basis for the research and \ndevelopment of AI and allied technologies.  \nII. The Ministry of IT & 
Telecom shall notify the establish ment  of an autonomous high -tech National \nAI Fund (NAIF) organization within 
six (6) months from the promulgation of this policy.  \nIII. The NAIF shall undertake  all the responsibilities and
implement guidelines as stipulated in this \npolicy or as directed by the Federal Government of Pakistan via the 
Ministry of IT & Telecom from \ntime to time.'
        ),
        0.9070492140470734
    ),
    (
        Document(
            metadata={'page': '11.0', 'source': 'National AI Policy Consultation Draft.pdf'},
            page_content='advanced technologies in AI and Data Science \nwhile incorporating PWDs and marginalized 
\nWomen through inclusion in Digis kills.pk \nProgram and other such initiatives in the \nprovinces.  By 2024  \n2.
AI Market \nEnablement  Target -5 National AI Fund  The Ignite Technology Fund is mandated to \nbroadly support R&D
Initiatives, \nEntrepreneurship, and Skill Development in \nICTs.  \nThere is no special provision for Hi -Tech, 
\nespecially AI and allied Technologies, at \npresent.  To establish National AI Fund for supporting \nHi-Tech 
initiatives in AI and Allied \nTechnologies nationwide as a spin -off of \nIgnite Technology Fund.  \n By 2023  \n 
\nTarget -6 Center of \nExcellence in AI & \nAllied Technologies  National Centers for AI are playing an 
\nessential role in applied R&D in AI at leading \nacademic institutions for higher education \nin Karachi, Lahore,
and Islamabad.  \nSino -Pak Center for Artificial Int elligence at \nPAF-IAST, Haripur, is also making tangible'
        ),
        0.8685352441807275
    ),
    (
        Document(
            metadata={'page': '6.0', 'source': 'National AI Policy Consultation Draft.pdf'},
            page_content='register  notable efforts  through public/private interventions as follows : \n• 
Presidential Initiative for Artificial Intelligence & Computing ( PIAIC ) \n• Center for Artificial Intelligence 
and Computing ( CENTAIC)  \n• National Center of Artificial Intelligence (NCAI)   \n• Sino -Pak Center for 
Artificial Intelligence  (SPCAI)  \nThese  timely  initiatives have performed well in their domains and need 
continuity to achieve long -term \ntargets. To achieve  the AI-accelerated  socio -economic development urgently 
required by Pakistan and its \nactual impact on the GDP of the country , a national strategy for AI adoption is 
critical . The AI policy provides \na holistic structure , by the societal challenges regarding AI adoption can be 
addressed by establishing the \nnecessary  ecosystem with specialized initiatives  and organizational 
responsibilities. The policy proposes \nthe National AI fund as a perpetual and central fund to support the 
proposed interventions .'
        ),
        0.8550270507206236
    )
]

In [31]:
# deleting the collection
# astra_vector_store.delete_collection()